In [62]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.baseline_only import BaselineOnly
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [2]:
movies = pd.read_csv('C:\\jupyter\\recommended system\\ml-latest\\movies.csv')
ratings = pd.read_csv('C:\\jupyter\\recommended system\\ml-latest\\ratings.csv')

In [3]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|Drama|Horror|Sci-Fi


In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753439,283228,8542,4.5,1379882795
27753440,283228,8712,4.5,1379882751
27753441,283228,34405,4.5,1379882889
27753442,283228,44761,4.5,1354159524


In [18]:
#на полном датасете не получилось сделать рекомендацию, возьмем только первые 100 000 записей или 1041 пользователей
r = ratings[:100051]
r

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
100046,1041,2599,4.0,939447991
100047,1041,2628,1.0,939447636
100048,1041,2657,4.0,939447778
100049,1041,2700,5.0,939449011


In [19]:
%%time
#создаем общую таблицу из которой понятно какой пользователь какому фильму поставил какую оценку
movies_with_ratings = movies.join(r.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

Wall time: 58.1 ms


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09


In [20]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09
...,...,...,...,...,...,...
148156,192081,Like Father (2018),Comedy,371.0,3.5,1.533570e+09
148213,192215,Forbidden Films (2014),Documentary,371.0,2.5,1.533570e+09
148215,192219,My Big Gay Italian Wedding (2018),Comedy,371.0,1.5,1.533570e+09
148218,192225,Redwall The Movie (2000),Animation|Children|Comedy|Fantasy,371.0,4.0,1.533570e+09


In [21]:
%%time
#создаем датасет для surprise
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

Wall time: 2.99 ms


In [22]:
dataset

,uid,iid,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0
...,...,...,...
148156,371.0,Like Father (2018),3.5
148213,371.0,Forbidden Films (2014),2.5
148215,371.0,My Big Gay Italian Wedding (2018),1.5
148218,371.0,Redwall The Movie (2000),4.0


In [31]:
%%time
#создаем данные для surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

Wall time: 87.7 ms


In [32]:
%%time
trainset, testset = train_test_split(data, test_size=0.20, random_state = 40)

Wall time: 126 ms


### Попробуем различные алгоритмы для построения рекомендательной системы

**KNNWithMeans**

In [41]:
%%time
#проверим разные similarities module при KNNWithMeans
similarities = ['msd', 'cosine', 'pearson', 'pearson_baseline']

for sim in similarities: 
    algo = KNNWithMeans(k=50, sim_options={'name': sim, 'user_based': False})
    algo.fit(trainset)
    test_pred = algo.test(testset)
    print(f'при {sim} rmse = {accuracy.rmse(test_pred)} \n' )

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9695
при msd rmse = 0.9694510702718743 

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9837
при cosine rmse = 0.9836570916595527 

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9821
при pearson rmse = 0.9821358599039492 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9605
при pearson_baseline rmse = 0.9605030538255869 

Wall time: 1min 2s


**Лучше всего резульаты получились при использовании "mds" или 'pearson_baseline' в качестве similarity matrix**

In [42]:
%%time
#проверим user-based параметр
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)
test_pred = algo.test(testset)
print(accuracy.rmse(test_pred))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9725
0.9724823482905187
Wall time: 2.29 s


**Влияние этого параметра незначительное при неизменности остальных параметров, но может существенно влиять при изменении метода или других параметров, так что стоит оставить user_based = False**

In [44]:
%%time
#проверим разные К
k_list = [30, 40, 50, 75]

for k in k_list: 
    algo = KNNWithMeans(k = k, sim_options={'name': 'pearson_baseline', 'user_based': False})
    algo.fit(trainset)
    test_pred = algo.test(testset)
    print(f'при k = {k} rmse = {accuracy.rmse(test_pred)} \n' )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9604
при k = 30 rmse = 0.9604209817853205 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9605
при k = 40 rmse = 0.9604605585031702 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9605
при k = 50 rmse = 0.9605030538255869 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9607
при k = 75 rmse = 0.960655464756074 

Wall time: 59.5 s


**Лучше всего результаты получились при k = 30. Однако rmse всё равно еще высокая, возможно смена алгоритма поможет**

In [48]:
%%time
algo = KNNBasic(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9859
при pearson_baseline rmse = 0.9858779148968283
Wall time: 14.7 s


In [51]:
%%time
algo = KNNWithZScore(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9637
при pearson_baseline rmse = 0.9636897157351583
Wall time: 15.4 s


In [52]:
%%time
algo = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9278
при pearson_baseline rmse = 0.9278465706498475
Wall time: 14.6 s


При использовании KNNBaseline rmse упало до 0.92, подберем параметры

Проверим user_based сразу

In [53]:
%%time
#true false
algo = KNNBaseline(k=40, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9566
при pearson_baseline rmse = 0.956617563721608
Wall time: 2.68 s


In [54]:
%%time
#проверим разные К
k_list = [30, 40, 50, 75]

for k in k_list: 
    algo = KNNBaseline(k = k, sim_options={'name': 'pearson_baseline', 'user_based': False})
    algo.fit(trainset)
    test_pred = algo.test(testset)
    print(f'при k = {k} rmse = {accuracy.rmse(test_pred)} \n' )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9279
при k = 30 rmse = 0.9279491412589668 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9278
при k = 40 rmse = 0.9278465706498475 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9278
при k = 50 rmse = 0.9278162687517933 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9279
при k = 75 rmse = 0.9278647410638214 

Wall time: 1min 1s


In [56]:
%%time
#проверим разные min_K
k_n_list = [3,7,10,15]

for k_n in k_n_list: 
    algo = KNNBaseline(k = 40, min_k = k_n, sim_options={'name': 'pearson_baseline', 'user_based': False})
    algo.fit(trainset)
    test_pred = algo.test(testset)
    print(f'при min_k = {k_n} rmse = {accuracy.rmse(test_pred)} \n' )

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9230
при min_k = 3 rmse = 0.9229578773889838 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9206
при min_k = 7 rmse = 0.9206262009371999 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9193
при min_k = 10 rmse = 0.91929693493471 

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9192
при min_k = 15 rmse = 0.9192284490610999 

Wall time: 56.1 s


In [61]:
#Попробуем вместо asl использовать sgd
%%time
algo = KNNBaseline(k=40, min_k = 10, sim_options={'name': 'pearson_baseline', 'user_based': False}, \
                   bsl_options = {'method': 'sgd', 'learning_rate': .00005})
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)} \n' )

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9504
при pearson_baseline rmse = 0.9504086736350296 

Wall time: 14.6 s


In [64]:
%%time
algo = BaselineOnly()
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)} \n' )

Estimating biases using als...
RMSE: 0.9453
при pearson_baseline rmse = 0.945306986443032 

Wall time: 292 ms


In [65]:
from surprise.prediction_algorithms.slope_one import SlopeOne

In [66]:
%%time
algo = SlopeOne()
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)} \n' )

<timed exec>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


RMSE: 0.9847
при pearson_baseline rmse = 0.9847037836793147 

Wall time: 7.28 s


In [67]:
from surprise.prediction_algorithms.matrix_factorization import SVD, SVDpp

In [68]:
%%time
algo = SVD()
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)} \n' )

RMSE: 0.9447
при pearson_baseline rmse = 0.9447449519349753 

Wall time: 4.29 s


In [69]:
%%time
algo = SVDpp()
algo.fit(trainset)
test_pred = algo.test(testset)
print(f'при {sim} rmse = {accuracy.rmse(test_pred)} \n' )

RMSE: 0.9339
при pearson_baseline rmse = 0.9339332176466267 

Wall time: 5min 58s
